# Wyzerr Editor Dashboard

In [1]:
from pymongo import MongoClient
import pymongo
import pandas as pd
from ipywidgets import interact, interactive, fixed
import datetime
import warnings
import ipywidgets as widgets
from IPython.display import display

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 999)

In [2]:
from read_db_surveys import *
from join_db_surveys import *
from read_db_responses import *
from join_db_responses import *
from get_metric import *
from get_results_for_survey import *
from read_db_account import *
from get_time_serial_signup import *
from get_activation import *

In [3]:
client = MongoClient('mongodb://localhost:27017/')

In [4]:
surveys = read_collection_surveys(client)
pages = read_collection_pages(client)
questions = read_collection_questions(client)
options = read_collection_options(client)

surveys_complete = join_db_surveys(options, questions, pages, surveys)

surveys_complete = surveys_complete[['accountId', 'createdAt', 'name']].drop_duplicates()
surveys_complete = surveys_complete.set_index(['accountId', 'name'])
surveys_complete = surveys_complete.sort_values(by = ['createdAt'])

In [5]:
responses_per_survey = get_responses_per_survey(client)
smartform_usage = get_smartform_usage(client)
account_activation = get_activation_time(client)

In [6]:
def account_metrics():
    result = {}
    result['total_num_of_account'] = len(account_activation)
    result['num_of_premium_account'] = account_activation['type'].value_counts()['premium']
    result['num_of_free_account'] = account_activation['type'].value_counts()['free']
    return pd.DataFrame.from_dict(result, orient='index')

## 1 Metrics, Data, and Data export

In [7]:
w = widgets.ToggleButtons(
    options=['account', 'survey', 'response', 'smartform'],
    description='SaaS',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
#     icon='check'
)

w1 = widgets.ToggleButtons(
    options=['metrics', 'data', 'data_download'],
    description='Options',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
#     icon='check'
)
@interact(w = w, w1 = w1, __manual=True)
def click_me(w, w1):
    if w == 'account':
        if w1 == 'metrics':
            display(account_metrics())
        elif w1 == 'data':
            display(account_activation)
        elif w1 == 'data_download':
            account_activation.to_csv('./data_export/account_data.csv')
            print 'the data is created already'
            
            
    elif w == 'survey':
        if w1 == 'metrics':
            print ('number of active surveys: %d' % len(surveys_complete))
        elif w1 == 'data':
            display(surveys_complete)
        elif w1 == 'data_download':
            surveys_complete.to_csv('./data_export/surveys.csv')
            print 'the data is created already'
        
        
        
    elif w == 'response':
        if w1 == 'metrics':
            print 'currently no metrics'
        elif w1 == 'data':
            display(responses_per_survey)
        elif w1 == 'data_download':
            responses_per_survey.to_csv('./data_export/survey_responses.csv')
            print 'the data is created already'
        
        
        
    elif w == 'smartform':
        if w1 == 'data_download':
            smartform_usage.to_csv('./data_export/smartform_usage.csv')
            print 'the data is created already'
        else:
            display(smartform_usage)
    
    

## 2 Graph

In [8]:
accounts = read_account_accounts(client)
accounts = accounts[['date', 'email']].drop_duplicates()
accounts.reset_index(inplace = True, drop = True)
accounts['date'] = UTC_to_EST(accounts['date'])

In [9]:
survey_trend = surveys[['createdAt', 'name']].drop_duplicates()
survey_trend.reset_index(drop = True, inplace = True)

In [10]:
responses = read_collection_responses(client)

responses_trend = responses[['_id', 'beginTime']].drop_duplicates()
responses_trend.reset_index(drop = True, inplace = True)

In [11]:
w2 = widgets.ToggleButtons(
    options=['signup', 'survey', 'response'],
    description='Trend',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
#     icon='check'
)

w3 = widgets.ToggleButtons(
    options=['hour', 'day'],
    description='Options',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
#     icon='check'
)
@interact(w2 = w2, w3 = w3, __manual=True)
def make_a_graph(w2, w3):
    if w2 == 'signup':
        if w3 == 'hour':
            editor_signup_over_time(accounts['date'], 'hour')
        elif w3 == 'day':
            editor_signup_over_time(accounts['date'], 'day')
    
    elif w2 == 'survey':
        if w3 == 'hour':
            editor_signup_over_time(survey_trend['createdAt'], 'hour')
        elif w3 == 'day':
            editor_signup_over_time(survey_trend['createdAt'], 'day')
            
    elif w2 == 'response':
        if w3 == 'hour':
            editor_signup_over_time(responses_trend['beginTime'], 'hour')
        elif w3 == 'day':
            editor_signup_over_time(responses_trend['beginTime'], 'day')

plot by day
